In [216]:
from typing import List, Tuple
import zipfile

import gensim.models
from navec import Navec
import numpy as np
import pandas as pd
import pymorphy3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import urllib.request

In [8]:
SEED = 42

In [71]:
def split(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    global SEED
    train_val_df, test_df = train_test_split(
        df, test_size=0.2, random_state=SEED, stratify=df["label"].values
    )
    train_df, val_df = train_test_split(
        train_val_df,
        test_size=0.2 * df.shape[0] / train_val_df.shape[0],
        random_state=SEED,
        stratify=train_val_df["label"].values,
    )
    return train_df, val_df, test_df


def tokenize(text: str) -> List[str]:
    return text.split(" ")

In [10]:
tokens_df = pd.read_csv("../data/hw1/processed_df.csv")
train_df, val_df, test_df = split(tokens_df)

In [11]:
train_df

,filtered_text,label
69234,экономика великобритании году сократится проце...,Экономика
36160,в минобороны россии назвали эффективность амер...,Силовые структуры
7065,американский боксер мохаммед али госпитализиро...,Спорт
53795,на окраине мурманска вторник обнаружен подполь...,Россия
63444,пользователь яндекса требует взыскать поискови...,Интернет и СМИ
...,...,...
45054,до конца года будут проведены учебно-боевых пу...,Силовые структуры
49047,комиссия российской академии наук изучив прете...,Россия
87334,компания ikea выпустила набор рецептов котором...,Интернет и СМИ
44320,в одном мурманских скверов июня года откроют с...,69-я параллель


## Обучение собственных эмбеддингов word2vec:
Обоснуем выбор некоторых гиперпараметров.

- vector_size - есть внутреннее ощущение, что слишком большие эмбеддинги могут переобучиться на этой задаче, поэтому попробуем начать с 250
- window - в задаче классификации текстов нам важно получать контекстную близость, а не синонимическую
- min_count - упрощаем обучение уменьшая маловстречаемые слова
- sg - Skip Gramm, чтобы быстрее учиться
- negative - аналогично, на каждой итерации вместо софтмакса по всему датасету считаем только 5 слов
- epochs - чем больше датасет, тем больше эпох можем поставить (обычно 10-50), начнем с 30


In [15]:
train_data = train_df["filtered_text"].apply(tokenize).values.tolist()

In [16]:
model = gensim.models.Word2Vec(
    sentences=train_data,
    vector_size=250,
    window=10,
    min_count=5,
    sg=1,
    negative=5,
    epochs=30,
    seed=SEED,
)

In [146]:
model.save("w2v_250_w10_mc5_sg_neg5_ep30")

In [77]:
print(" | ".join(train_df["label"].unique().tolist()))

Экономика | Силовые структуры | Спорт | Россия | Интернет и СМИ | Из жизни | Мир | Наука и техника | Дом | Бывший СССР | Культура | Ценности | Путешествия | Бизнес | Легпром | 69-я параллель | Крым | Культпросвет  | Библиотека


Попробуем оценить качество эмбеддингов, используя самые частотные

In [72]:
model.wv.most_similar("экономика")

[('ввп', 0.6906951665878296),
 ('рецессии', 0.6488956212997437),
 ('рецессию', 0.6266115307807922),
 ('замедлятся', 0.6121088862419128),
 ('адаптировалась', 0.5717689394950867),
 ('экономики', 0.5682619214057922),
 ('инфляция', 0.5424972176551819),
 ('экономический', 0.5400095582008362),
 ('рецессия', 0.5394936203956604),
 ('рецессией', 0.5338839292526245)]

In [89]:
model.wv.most_similar(negative=["экономика"])

[('нигерийского', 0.1339004635810852),
 ('прицельный', 0.12875449657440186),
 ('рыжего', 0.11170051246881485),
 ('палестинца', 0.11158581078052521),
 ('стрельбой', 0.10589572787284851),
 ('четверым', 0.10377943515777588),
 ('гаражного', 0.10017924010753632),
 ('пятерым', 0.09932998567819595),
 ('самары', 0.09752027690410614),
 ('красноярска', 0.09594501554965973)]

In [90]:
model.wv.most_similar("мвд")

[('гу', 0.7301331162452698),
 ('милиции', 0.6089017391204834),
 ('следственного', 0.6055797934532166),
 ('главка', 0.6005496382713318),
 ('внутренних', 0.5966971516609192),
 ('увд', 0.5913692712783813),
 ('оперативно-розыскного', 0.5893971920013428),
 ('говд', 0.5781435966491699),
 ('гувд', 0.5723111629486084),
 ('фсб', 0.5708879232406616)]

In [91]:
model.wv.most_similar(negative=["мвд"])

[('ямайка', 0.1073177233338356),
 ('дилан', 0.10620534420013428),
 ('melissa', 0.10451147705316544),
 ('консультировал', 0.10422379523515701),
 ('здоровой', 0.10061188787221909),
 ('подниматься', 0.1005881130695343),
 ('фазе', 0.09912239015102386),
 ('высказывают', 0.095782071352005),
 ('lake', 0.09563589841127396),
 ('охлаждении', 0.09526385366916656)]

In [83]:
model.wv.most_similar("футбол")

[('баскетбол', 0.5135613679885864),
 ('футболистов', 0.4858241677284241),
 ('атлетику', 0.48419076204299927),
 ('матч', 0.47358715534210205),
 ('играть', 0.47240549325942993),
 ('легии', 0.4598545730113983),
 ('футбольный', 0.45508161187171936),
 ('болельщики', 0.4550151824951172),
 ('расслабляться', 0.4545561671257019),
 ('футбола', 0.45173168182373047)]

In [92]:
model.wv.most_similar(negative=["футбол"])

[('фгупов', 0.12461049109697342),
 ('сертификации', 0.12004587799310684),
 ('подлежат', 0.1166456788778305),
 ('вывезены', 0.11573202162981033),
 ('имелся', 0.11476853489875793),
 ('согласованию', 0.11053688079118729),
 ('печатью', 0.11017366498708725),
 ('сложностях', 0.10659559071063995),
 ('отозвало', 0.10580680519342422),
 ('позволяло', 0.10374383628368378)]

Выглядит так, что ключевые слова разделены по категориям, поэтому параметры подобраны достаточно хорошо

In [ ]:
urllib.request.urlretrieve(
    "https://vectors.nlpl.eu/repository/20/184.zip", "news_upos_skipgram_300_5_2019.zip"
)
zip_path = "news_upos_skipgram_300_5_2019.zip"
extract_folder = "news_upos_skipgram_300_5_2019"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

('news_upos_skipgram_300_5_2019.zip',
 <http.client.HTTPMessage at 0x72fca26c6ed0>)

In [100]:
rusvec_model_path = "news_upos_skipgram_300_5_2019/model.bin"
rusvec_model = gensim.models.KeyedVectors.load_word2vec_format(
    rusvec_model_path, binary=True
)

In [103]:
rusvec_model.most_similar("мвд_NOUN")

[('гумвд_NOUN', 0.4122755229473114),
 ('уфсб_NOUN', 0.41181737184524536),
 ('гувд_NOUN', 0.392577588558197),
 ('адриан::ранкин-гэллоуэй_PROPN', 0.3685126304626465),
 ('алиевой_NOUN', 0.3653308153152466),
 ('возражалийский_ADJ', 0.3638455271720886),
 ('мчс::россия::раиса::копырина_PROPN', 0.3625434935092926),
 ('амра::юзбеков_PROPN', 0.36013513803482056),
 ('республиканца_NOUN', 0.3595658838748932),
 ('мвд::россия_PROPN', 0.35692664980888367)]

И теперь из navec

In [105]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2025-03-13 23:01:42--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Распознаётся storage.yandexcloud.net (storage.yandexcloud.net)… 213.180.193.243, 2a02:6b8::1d9
Подключение к storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 53012480 (51M) [application/x-tar]
Сохранение в: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50,56M  10,9MB/s    за 5,1s    

2025-03-13 23:01:47 (9,99 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ сохранён [53012480/53012480]



In [ ]:
path = "navec_hudlit_v1_12B_500K_300d_100q.tar"
navec_model = Navec.load(path)

In [131]:
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0
    return np.dot(vec1, vec2) / (norm1 * norm2)


def ineffective_most_similiar(
    navec_model: Navec, word: str, top_n: int = 10
) -> List[Tuple[str, float]]:
    if word not in navec_model:
        raise ValueError(f"Слова {word} нет в модели")

    word_vector = navec_model[word]
    similarities = {}
    for other_word in navec_model.vocab.words:
        other_vector = navec_model[other_word]
        other_norm = np.linalg.norm(other_vector)

        if other_norm == 0:
            continue

        similarities[other_word] = cosine_similarity(word_vector, other_vector)

    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]

In [133]:
ineffective_most_similiar(navec_model, "мвд")

[('мвд', 1.0),
 ('фсб', 0.76623964),
 ('кгб', 0.7264482),
 ('мгб', 0.69483584),
 ('гувд', 0.68139136),
 ('нквд', 0.6729191),
 ('госбезопасности', 0.6632358),
 ('увд', 0.6568187),
 ('гру', 0.64401966),
 ('прокуратуры', 0.6382184)]

In [138]:
vector_size = navec_model.pq.dim
gensim_from_navec = gensim.models.KeyedVectors(vector_size)
words, vectors = [], []
for word in navec_model.vocab.words:
    vec = navec_model[word]
    if np.linalg.norm(vec) > 0:
        words.append(word)
        vectors.append(vec)

gensim_from_navec.add_vectors(words, np.array(vectors))

Есть ощущение, что вектора из navec и rusvector больше склонны показывать функциональную близость - т.е. слова, которые являются почти синонимами

Обучим на каждом виде эмбеддингов модель логистической регрессии

In [252]:
def create_features(
    data: pd.DataFrame,
    emb_model: gensim.models.Word2Vec,
    use_tag: bool = False,
    is_kv: bool = False,
    emb_dim: int = 300,
) -> np.ndarray:
    if use_tag:
        morph = pymorphy3.MorphAnalyzer()
    X = np.empty((0, emb_dim))
    for text in data["filtered_text"].values:
        if use_tag:
            pos_tags = [morph.parse(word)[0].tag.POS for word in text.split(" ")]
        avg_emb = np.empty((0, emb_dim))
        for i, token in enumerate(text.split(" ")):
            token = f"{token}_{pos_tags[i]}" if use_tag else token
            embs_storage = emb_model if is_kv else emb_model.wv
            if token in embs_storage:
                emb = embs_storage[token]
                avg_emb = np.vstack((avg_emb, emb))
        if len(avg_emb) > 0:
            avg_emb = np.mean(avg_emb, axis=0).reshape(1, emb_dim)
        else:
            avg_emb = np.zeros((1, emb_dim))
        X = np.vstack((X, avg_emb))
    return X


In [ ]:
custom_train_features = create_features(train_df, model, emb_dim=250)
rusvec_train_features = create_features(
    train_df, rusvec_model, use_tag=True, is_kv=True, emb_dim=300
)
navec_train_features = create_features(
    train_df, gensim_from_navec, is_kv=True, emb_dim=300
)

In [211]:
for fpath, features in zip(
    ["custom_train", "rusvec_train", "navec_train"],
    [custom_train_features, rusvec_train_features, navec_train_features],
):
    with open(f"../data/hw2/{fpath}.npy", "wb") as f:
        np.save(f, features)

In [ ]:
custom_val_features = create_features(val_df, model, emb_dim=250)
rusvec_val_features = create_features(
    val_df, rusvec_model, use_tag=True, is_kv=True, emb_dim=300
)
navec_val_features = create_features(val_df, gensim_from_navec, is_kv=True, emb_dim=300)

In [ ]:
custom_test_features = create_features(test_df, model, emb_dim=250)
rusvec_test_features = create_features(
    test_df, rusvec_model, use_tag=True, is_kv=True, emb_dim=300
)
navec_test_features = create_features(
    test_df, gensim_from_navec, is_kv=True, emb_dim=300
)

In [255]:
def train_logreg(X: np.ndarray, y: np.ndarray) -> LogisticRegression:
    logreg = LogisticRegression(max_iter=2000)
    logreg.fit(X, y)
    return logreg


y = train_df["label"].values
# custom_fs_logreg = train_logreg(custom_train_features, y)
rusvec_fs_logreg = train_logreg(rusvec_train_features, y)
# navec_fs_logreg = train_logreg(navec_train_features, y)

In [256]:
rusvec_vp = rusvec_fs_logreg.predict(rusvec_train_features)
print(classification_report(train_df["label"].values, rusvec_vp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.83      0.19      0.31       103
       Библиотека       0.00      0.00      0.00         5
           Бизнес       0.51      0.12      0.19       601
      Бывший СССР       0.73      0.65      0.69      4333
              Дом       0.76      0.70      0.73      1764
         Из жизни       0.57      0.47      0.52      2241
   Интернет и СМИ       0.65      0.56      0.60      3625
             Крым       0.71      0.09      0.16        54
    Культпросвет        0.00      0.00      0.00        28
         Культура       0.81      0.81      0.81      4366
          Легпром       0.00      0.00      0.00         9
              Мир       0.73      0.79      0.76     11092
  Наука и техника       0.75      0.75      0.75      4303
      Путешествия       0.69      0.42      0.53       520
           Россия       0.70      0.79      0.74     13015
Силовые структуры       0.50      0.21      0.29      1

In [257]:
custom_vp = custom_fs_logreg.predict(custom_train_features)
print(classification_report(train_df["label"].values, custom_vp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.90      0.25      0.39       103
       Библиотека       0.00      0.00      0.00         5
           Бизнес       0.58      0.21      0.31       601
      Бывший СССР       0.81      0.82      0.82      4333
              Дом       0.85      0.80      0.82      1764
         Из жизни       0.67      0.61      0.64      2241
   Интернет и СМИ       0.76      0.70      0.73      3625
             Крым       1.00      0.09      0.17        54
    Культпросвет        0.00      0.00      0.00        28
         Культура       0.87      0.88      0.88      4366
          Легпром       0.00      0.00      0.00         9
              Мир       0.80      0.84      0.82     11092
  Наука и техника       0.84      0.84      0.84      4303
      Путешествия       0.77      0.61      0.68       520
           Россия       0.77      0.83      0.80     13015
Силовые структуры       0.63      0.35      0.45      1

In [ ]:
rusvec_vp = rusvec_fs_logreg.predict(rusvec_val_features)
print(classification_report(val_df["label"].values, rusvec_vp, zero_division=0))

In [ ]:
navec_vp = navec_fs_logreg.predict(navec_val_features)
print(classification_report(val_df["label"].values, navec_vp, zero_division=0))